In [17]:
import pandas as pd
import numpy as np
import pandas_profiling # pip install pandas_profiling

taxiCancellationScore = pd.read_csv("Kaggle_YourCabs_score.csv")
taxiCancellationTraining = pd.read_csv("Kaggle_YourCabs_training.csv")

In [18]:
profile = pandas_profiling.ProfileReport(taxiCancellationTraining)
display(profile)

In [19]:
taxiCancellationTraining.columns

Index(['id', 'user_id', 'vehicle_model_id', 'package_id', 'travel_type_id',
       'from_area_id', 'to_area_id', 'from_city_id', 'to_city_id', 'from_date',
       'to_date', 'online_booking', 'mobile_site_booking', 'booking_created',
       'from_lat', 'from_long', 'to_lat', 'to_long', 'Car_Cancellation',
       'Cost_of_error'],
      dtype='object')

In [20]:
taxiCancellationTraining.isnull().sum()

id                         0
user_id                    0
vehicle_model_id           0
package_id             35881
travel_type_id             0
from_area_id              88
to_area_id              9138
from_city_id           27086
to_city_id             41843
from_date                  0
to_date                17890
online_booking             0
mobile_site_booking        0
booking_created            0
from_lat                  93
from_long                 93
to_lat                  9138
to_long                 9138
Car_Cancellation           0
Cost_of_error              0
dtype: int64

In [21]:
taxiCancellationScore.isnull().sum()

id                         0
user_id                    0
vehicle_model_id           0
package_id              8215
travel_type_id             0
from_area_id              20
to_area_id              2150
from_city_id            6288
to_city_id              9635
from_date                  0
to_date                 4127
online_booking             0
mobile_site_booking        0
booking_created            0
from_lat                  20
from_long                 20
to_lat                  2150
to_long                 2150
Unnamed: 18            10000
Unnamed: 19            10000
dtype: int64

In [22]:
feature_cols = ['id', 'user_id', 'vehicle_model_id', 'package_id', 'travel_type_id',
                'from_area_id', 'to_area_id', 'from_city_id', 'to_city_id', 'from_date',
       'to_date', 'online_booking', 'mobile_site_booking', 'booking_created',
       'from_lat', 'from_long', 'to_lat', 'to_long']
X = taxiCancellationTraining[feature_cols] # Features
y = taxiCancellationTraining.Car_Cancellation # Target variable

In [23]:
X_test = taxiCancellationScore[feature_cols] # Features

## 4 Models

In [24]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout

#Logistic Regresstion

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
logreg.fit(X, y)


#Deep Learning Keras Model
#relu -linear
#sigmoid function for final layer
#Dropout 50%
#Dimension input features column
#Output binary classification 0 and 1
model = Sequential()
model.add(Dense(124, activation='relu', input_dim = X.shape[1]))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.summary()
#Using binary_crossentropy
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])



#KNN
n_neigh = 3
from sklearn.neighbors import KNeighborsClassifier
neighbor = KNeighborsClassifier(n_neigh)
neighbor.fit(X, y) 


#Naive Bayes
from sklearn.naive_bayes import GaussianNB
gaus = GaussianNB()
clf.fit(X, Y)

ValueError: could not convert string to float: '11/24/2013 14:50'

# Prediction

# Performance of Each Model

# visualization